<a href="https://colab.research.google.com/github/TomasPastore/aprendizaje_automatico_tp/blob/main/tp_AA_grupo_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import timeit
from sklearn.metrics import roc_auc_score

# Lectura del dataset


# Sorry quizas rompi algo , ceno y en un rato lo arreglo

In [ ]:
url = 'https://raw.githubusercontent.com/aprendizaje-automatico-dc-uba-ar/material/main/tp/01_aprendizaje_supervisado/datos/minions_publicos.csv'
minions_dataset = pd.read_csv(url)

y = minions_dataset.iloc[:,-1:]
print(y[y["target"] == 1])

del minions_dataset[minions_dataset.columns[-1]]
print("\n Features")
minions_dataset

     target
0         1
1         1
2         1
3         1
5         1
..      ...
483       1
490       1
491       1
497       1
499       1

[154 rows x 1 columns]

 Features


,v000,v001,v002,v003,v004,v005,v006,v007,v008,v009,...,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199
0,0.203777,-7.278026,2.756077,0.412740,-0.116626,0.414236,6.218588,0.018764,0.209555,0.005660,...,-8.551783,-5.323863,2.004479,-0.065859,-0.877748,0.221820,0.103884,-0.219372,2.179068,0.578645
1,1.084936,0.328886,4.545345,0.678987,0.003042,-0.807901,-13.653907,-0.654958,-0.860854,-1.017529,...,1.195129,-20.574463,0.119288,0.505041,0.922128,0.405464,1.886947,0.407544,1.431193,0.159015
2,0.769527,-1.213339,-1.821305,-0.816333,3.059691,-0.583517,22.961234,-1.034725,-0.039125,0.983415,...,2.926464,14.608653,-0.456923,-0.647099,-0.565453,-0.137190,-0.232586,-1.420808,-0.843064,0.274254
3,-0.094123,-0.567694,1.747513,0.033398,7.415277,-0.756049,-1.021780,-0.662921,-0.671345,-0.159661,...,4.750111,21.559404,0.188952,0.137507,1.186905,0.955207,0.589803,1.509390,0.992935,1.875483
4,0.116567,-1.551208,9.343513,-1.785618,-2.651805,0.955870,-0.952141,-0.854849,-0.679621,0.682699,...,0.364826,24.729498,-0.585663,-1.551728,0.239188,-0.882897,0.129474,0.019597,-0.322608,0.302670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.288139,-1.396761,-4.263799,0.749690,1.721721,-0.344043,19.469950,0.662457,0.557387,-0.190842,...,1.589264,17.684302,-1.333075,0.388253,1.314865,0.244731,-0.051043,0.768067,-1.622517,-0.276086
496,0.153794,-1.877765,0.514750,3.191452,2.515170,0.214678,1.100869,-0.641812,-0.192485,2.340703,...,-0.849789,-12.417970,0.906713,-0.066838,1.210734,1.491540,1.586784,-0.445346,1.701997,-0.326600
497,0.128327,-2.151517,-6.010139,-5.710254,-2.606002,-0.326701,-3.537265,-0.786013,-0.985565,0.120138,...,-10.669278,-7.494708,0.520567,-1.523764,0.714171,0.222749,-0.126531,-0.074166,-0.752975,-1.750938
498,0.907801,2.899501,-2.170869,1.386165,4.658550,0.172423,2.034290,0.563519,-1.294684,0.853006,...,0.691517,-18.558853,1.667956,0.697122,2.236943,-0.629723,0.969926,-1.121823,-0.413634,0.321612


Esto nos indica que hay 154 de clase positiva y el resto de clase negativa. Es decir, está desbalanceado en aproximadamente 30/70. Tenemos que tener en cuenta entonces esto a la hora de splitear los datos.

# 1) Separación de datos

Para la separación de datos de evaluación, lo primero que tuvimos en cuenta fue evaluar la distribucion de etiquetas de los minions entrevistados. Al revisar esto notamos que hay un desbalance entre minions que fueron aceptados vs aquellos que no (aproximadamente fueron aceptados el 30% de los postulantes). Por lo tanto es primordial que mantengamos la proporción entre estas dos clases cuando hagamos los _k-folds_, ya que de no realizarlo nuestro predictor no se entrenaría con las proporciones correctas de los datos. 

En segundo lugar tenemos que tener en cuenta la cantidad de candidatos que tenemos. Solamente 500 postulados resulta una base de datos más bien reducida, por lo que no tenemos margen a la hora de decidir nuestra estrategia a la hora de separación de datos. Lo que nuestro grupo propone es realizar una separación del 10% para utilizar en la etapa de evaluación final, manteniendo el 90% restante para el desarrollo de nuestros modelos (todo esto haciendo _stratified split_ para que el desbalance de datos no nos perjudique a futuro).

#########################
Revisar juntos, cuando hacemos el train_test_split se elije al azar, se mantiene las proporciones de las clases pero se hace al azar, no estamos considerando el desbalanceo. mas datos para eval??

In [ ]:
from sklearn.model_selection import train_test_split

X_dev, X_eval, y_dev, y_eval = train_test_split(minions_dataset, y, shuffle=True, random_state=4, test_size=0.1) # quedan 15 positivos para eval

print(f"X_train dimensión: {X_dev.shape}")
print(f"y_train dimensión: {y_dev.shape}")

print(f"X_test dimensión: {X_eval.shape}")
print(f"y_test dimensión: {y_eval.shape}")


X_train dimensión: (450, 200)
y_train dimensión: (450, 1)
X_test dimensión: (50, 200)
y_test dimensión: (50, 1)


# 2) Construcción de modelos

# 2.1) Árbol default con max_height 3

In [ ]:
from sklearn.tree import DecisionTreeClassifier

arbol_gini_3 = DecisionTreeClassifier(max_depth=3)

## 2.2) Iteradores de cross validation

In [ ]:

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

balanced_k_fold = StratifiedKFold(n_splits=5)

# Validar si podemos usar esto porque el enunciado dice k fold
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=4)

X_train_cv = []
y_train_cv = []
X_test_cv = []
y_test_cv = []

for train_index, test_index in sss.split(X_dev, y_dev):
    X_train = X_dev.iloc[train_index]
    X_test = X_dev.iloc[test_index]
    y_train = y_dev.iloc[train_index]
    y_test = y_dev.iloc[test_index]
    
    X_train_cv.append(X_train)
    X_test_cv.append(X_test)
    y_train_cv.append(y_train)
    y_test_cv.append(y_test)


## 2.2) Métricas

In [ ]:
from sklearn import metrics

# Scoring metrics

def tn(y, y_pred): return metrics.confusion_matrix(y, y_pred)[0, 0];
def fp(y, y_pred): return metrics.confusion_matrix(y, y_pred)[0, 1];
def fn(y, y_pred): return metrics.confusion_matrix(y, y_pred)[1, 0];
def tp(y, y_pred): return metrics.confusion_matrix(y, y_pred)[1, 1];
def specificity(y, y_pred): return tn(y, y_pred)/(tn(y, y_pred) + fp(y,y_pred));
def precision(y, y_pred): return tp(y, y_pred)/(tp(y, y_pred) + fp(y, y_pred));
def recall(y, y_pred): return tp(y, y_pred)/(tp(y, y_pred) + fn(y, y_pred));
def f1(y, y_pred): return 2 * precision(y, y_pred) * recall(y, y_pred) / (precision(y, y_pred) + recall(y_pred))

scoring = {#'precision': metrics.make_scorer(precision), 
           #'recall': metrics.make_scorer(recall),
           #'specificity': metrics.make_scorer(specificity),
           #'tp': metrics.make_scorer(tp),
           #'tn': metrics.make_scorer(tn),
           #'fp': metrics.make_scorer(fp),
           #'fn': metrics.make_scorer(fn),
           #'f1_score': metrics.make_scorer(f1),
           'roc_auc': metrics.make_scorer(metrics.roc_auc_score),
           'accuracy': 'accuracy'
           }


## 2.2) Árbol default (gini) altura 3 + Crossvalidation K=5 + Accuracy y AUC 

Integramos las 3 celdas anteriores para correr los folds y calcular las metricas para un classificador

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline

# Runs crossvalidation with one fixed configuration of parameters.
# for each classifier in the values of classifiers dictionary (param)
# scoring is a dictionary with the metrics we want to calculate accross iterations
# cv is a cross validation iterator

# TODO modificar para que imprima el global, el promedio etc, cada fold
def cv_with_metrics(classifiers, X, y, scoring, cv):
    for clf_name, clf in classifiers.items():
        print('Crossvalidating with {0}'.format(clf_name))
        pipeline_clf = make_pipeline(preprocessing.StandardScaler(), clf)
        cv_results = cross_validate(pipeline_clf, X, y=y, scoring=scoring, cv=cv, n_jobs=-1)
        print('Scoring mean across folds for {0}...'.format(clf_name))
        for key, v in cv_results.items():
            if 'test' in key:
                print('\t{k} --> {m}'.format(k=key, m=np.mean(cv_results[key])))


In [ ]:
# Corremos el K fold cross validation para una configuracion fija
classifiers_to_test = {'Árbol de Decisión <default, max_h=3 > ': arbol_gini_3 }
cv_with_metrics(classifiers_to_test, X_dev, y_dev, scoring, cv=balanced_k_fold)


Crossvalidating with Árbol de Decisión <default, max_h=3 > 
Scoring mean across folds for Árbol de Decisión <default, max_h=3 > ...
	test_roc_auc --> 0.5754949650110941
	test_accuracy --> 0.6644444444444444


**Resultados**<table>
      <thead>
      <tr>
      <th align="center">Permutación</th>
      <th>Accuracy (training)</th>
      <th>Accuracy (validación)</th>
      <th>AUC ROC (training)</th>
      <th>AUC ROC (validación)</th>
      </tr>
      </thead>
      <tbody>
      <tr>
      <td align="center">1</td>
      <td>0,769</td>
      <td>0.722</td>
      <td>0.815</td>
      <td>0.759</td>
      </tr>
      <tr>
      <td align="center">2</td>
      <td>0.806</td>
      <td>0.667</td>
      <td>0.774</td>
      <td>0.622</td>
      </tr>
      <tr>
      <td align="center">3</td>
      <td>0.841</td>
      <td>0.689</td>
      <td>0.837</td>
      <td>0.614</td>
      </tr>
      <tr>
      <td align="center">4</td>
      <td>0.786</td>
      <td>0.6</td>
      <td>0.757</td>
      <td>0.493</td>
      </tr>
      <tr>
      <td align="center">5</td>
      <td>0.839</td>
      <td>0.711</td>
      <td>0.841</td>
      <td>0.655</td>
      </tr>
      <tr>
      <td align="center">Global</td>
      <td>0.678</td>
      <td>0.808</td>
      <td>0.817</td>
      <td>0.629</td>
      </tr>
      </tbody>
      </table>

#2.3) Parameter grid sobre el arbol

In [ ]:
# gini (default) con max depth 3, 5 e inf (default)
# entropy con max depth 3, 5 e inf

from sklearn.model_selection import ParameterGrid

def parameter_grid(grid):
  param_grid = ParameterGrid(grid)
  for config in param_grid:
    

In [ ]:
arboles_cv = []
y_pred_cv = []
AAA = 0
BBB = 0
CCC = 0
DDD = 0

for i in range(5):
  arbol = DecisionTreeClassifier(max_depth=3)
  arbol.fit(X_train_cv[i], y_train_cv[i])
  arboles_cv.append(arbol)

  # predecimos los valores para las instacias que no vimos
  y_pred = arbol.predict(X_test_cv[i])
  y_pred_cv.append(y_pred)
  y_pred_test = arbol.predict(X_train_cv[i])
  y_pred_cv.append(y_pred_test)

  # print(y_pred)


  #print(f"Accuracy sobre el test set: {np.mean(y_pred == y_eval)}") 
  print(f"Accuracy sobre el train set: {arbol.score(X_train_cv[i], y_train_cv[i])}")

  auc_train = roc_auc_score(y_pred_test, y_train_cv[i])
  print("AUC de la curva ROC sobre el train set:", auc_train)



  #print(f"Accuracy sobre el test set: {np.mean(y_pred == y_eval)}") 
  print(f"Accuracy sobre el test set: {arbol.score(X_test_cv[i], y_test_cv[i])}")

  auc_test = roc_auc_score(y_pred, y_test_cv[i])
  print("AUC de la curva ROC sobre el test set:", auc_test)


  AAA += arbol.score(X_test_cv[i], y_test_cv[i])
  BBB += auc_test
  CCC += arbol.score(X_train_cv[i], y_train_cv[i])
  DDD += auc_train


print("accuracy promedio de test:", AAA/5)
print("AUC promedio de test:", BBB/5)
print("accuracy promedio de train:", CCC/5)
print("AUC promedio de train:", DDD/5)



Accuracy sobre el train set: 0.7694444444444445
AUC de la curva ROC sobre el train set: 0.875
Accuracy sobre el test set: 0.7111111111111111
AUC de la curva ROC sobre el test set: 0.6722560975609756
Accuracy sobre el train set: 0.8055555555555556
AUC de la curva ROC sobre el train set: 0.7742650712947743
Accuracy sobre el test set: 0.6666666666666666
AUC de la curva ROC sobre el test set: 0.6223060344827586
Accuracy sobre el train set: 0.8416666666666667
AUC de la curva ROC sobre el train set: 0.8372960168450605
Accuracy sobre el test set: 0.6888888888888889
AUC de la curva ROC sobre el test set: 0.6148648648648648
Accuracy sobre el train set: 0.7861111111111111
AUC de la curva ROC sobre el train set: 0.7569875776397517
Accuracy sobre el test set: 0.6
AUC de la curva ROC sobre el test set: 0.4928571428571429
Accuracy sobre el train set: 0.8388888888888889
AUC de la curva ROC sobre el train set: 0.8408058372722682
Accuracy sobre el test set: 0.7111111111111111
AUC de la curva ROC sobre 

In [ ]:
arboles_cv = []
y_pred_cv = []
AAA = 0
CCC = 0

for i in range(5):
  arbol = DecisionTreeClassifier(max_depth=5)
  arbol.fit(X_train_cv[i], y_train_cv[i])
  arboles_cv.append(arbol)

  # predecimos los valores para las instacias que no vimos
  y_pred = arbol.predict(X_test_cv[i])
  y_pred_cv.append(y_pred)
  # print(y_pred)

  #print(f"Accuracy sobre el test set: {np.mean(y_pred == y_eval)}") 
  print(f"Accuracy sobre el test set: {arbol.score(X_test_cv[i], y_test_cv[i])}")

  AAA += arbol.score(X_test_cv[i], y_test_cv[i])
  CCC += arbol.score(X_train_cv[i], y_train_cv[i])

print("accuracy promedio de test:", AAA/5)
print("accuracy promedio de train:", CCC/5)

Accuracy sobre el test set: 0.7333333333333333
Accuracy sobre el test set: 0.6444444444444445
Accuracy sobre el test set: 0.6777777777777778
Accuracy sobre el test set: 0.6111111111111112
Accuracy sobre el test set: 0.6777777777777778
accuracy promedio de test: 0.6688888888888889
accuracy promedio de train: 0.9122222222222222


In [ ]:
arboles_cv = []
y_pred_cv = []
AAA = 0
CCC = 0

for i in range(5):
  arbol = DecisionTreeClassifier()
  arbol.fit(X_train_cv[i], y_train_cv[i])
  arboles_cv.append(arbol)

  # predecimos los valores para las instacias que no vimos
  y_pred = arbol.predict(X_test_cv[i])
  y_pred_cv.append(y_pred)
  # print(y_pred)

  #print(f"Accuracy sobre el test set: {np.mean(y_pred == y_eval)}") 
  print(f"Accuracy sobre el test set: {arbol.score(X_test_cv[i], y_test_cv[i])}")

  AAA += arbol.score(X_test_cv[i], y_test_cv[i])
  CCC += arbol.score(X_train_cv[i], y_train_cv[i])

print("accuracy promedio de test:", AAA/5)
print("accuracy promedio de train:", CCC/5)

Accuracy sobre el test set: 0.6888888888888889
Accuracy sobre el test set: 0.6555555555555556
Accuracy sobre el test set: 0.6666666666666666
Accuracy sobre el test set: 0.6333333333333333
Accuracy sobre el test set: 0.6777777777777778
accuracy promedio de test: 0.6644444444444444
accuracy promedio de train: 1.0


In [ ]:
arboles_cv = []
y_pred_cv = []
AAA = 0
CCC = 0

for i in range(5):
  arbol = DecisionTreeClassifier(criterion='entropy', max_depth=3)
  arbol.fit(X_train_cv[i], y_train_cv[i])
  arboles_cv.append(arbol)

  # predecimos los valores para las instacias que no vimos
  y_pred = arbol.predict(X_test_cv[i])
  y_pred_cv.append(y_pred)
  # print(y_pred)

  #print(f"Accuracy sobre el test set: {np.mean(y_pred == y_eval)}") 
  print(f"Accuracy sobre el test set: {arbol.score(X_test_cv[i], y_test_cv[i])}")

  AAA += arbol.score(X_test_cv[i], y_test_cv[i])
  CCC += arbol.score(X_train_cv[i], y_train_cv[i])

print("accuracy promedio de test:", AAA/5)
print("accuracy promedio de train:", CCC/5)

Accuracy sobre el test set: 0.7111111111111111
Accuracy sobre el test set: 0.6222222222222222
Accuracy sobre el test set: 0.6888888888888889
Accuracy sobre el test set: 0.6
Accuracy sobre el test set: 0.6444444444444445
accuracy promedio de test: 0.6533333333333333
accuracy promedio de train: 0.7861111111111111


In [ ]:
arboles_cv = []
y_pred_cv = []
AAA = 0
CCC = 0

for i in range(5):
  arbol = DecisionTreeClassifier(criterion='entropy', max_depth=5)
  arbol.fit(X_train_cv[i], y_train_cv[i])
  arboles_cv.append(arbol)

  # predecimos los valores para las instacias que no vimos
  y_pred = arbol.predict(X_test_cv[i])
  y_pred_cv.append(y_pred)
  # print(y_pred)

  #print(f"Accuracy sobre el test set: {np.mean(y_pred == y_eval)}") 
  print(f"Accuracy sobre el test set: {arbol.score(X_test_cv[i], y_test_cv[i])}")

  AAA += arbol.score(X_test_cv[i], y_test_cv[i])
  CCC += arbol.score(X_train_cv[i], y_train_cv[i])

print("accuracy promedio de test:", AAA/5)
print("accuracy promedio de train:", CCC/5)

Accuracy sobre el test set: 0.6555555555555556
Accuracy sobre el test set: 0.6111111111111112
Accuracy sobre el test set: 0.7
Accuracy sobre el test set: 0.6444444444444445
Accuracy sobre el test set: 0.6333333333333333
accuracy promedio de test: 0.6488888888888888
accuracy promedio de train: 0.8983333333333334


In [ ]:
arboles_cv = []
y_pred_cv = []
AAA = 0
CCC = 0

for i in range(5):
  arbol = DecisionTreeClassifier(criterion='entropy')
  arbol.fit(X_train_cv[i], y_train_cv[i])
  arboles_cv.append(arbol)

  # predecimos los valores para las instacias que no vimos
  y_pred = arbol.predict(X_test_cv[i])
  y_pred_cv.append(y_pred)
  # print(y_pred)

  #print(f"Accuracy sobre el test set: {np.mean(y_pred == y_eval)}") 
  print(f"Accuracy sobre el test set: {arbol.score(X_test_cv[i], y_test_cv[i])}")

  AAA += arbol.score(X_test_cv[i], y_test_cv[i])
  CCC += arbol.score(X_train_cv[i], y_train_cv[i])

print("accuracy promedio de test:", AAA/5)
print("accuracy promedio de train:", CCC/5)

Accuracy sobre el test set: 0.6111111111111112
Accuracy sobre el test set: 0.6222222222222222
Accuracy sobre el test set: 0.6666666666666666
Accuracy sobre el test set: 0.6555555555555556
Accuracy sobre el test set: 0.5888888888888889
accuracy promedio de test: 0.6288888888888888
accuracy promedio de train: 1.0


<table>
   <thead>
   <tr>
   <th align="center">Altura máxima</th>
   <th align="center">Criterio de corte</th>
   <th>Accuracy (training)</th>
   <th>Accuracy (validación)</th>
   </tr>
   </thead>
   <tbody><tr>
   <td align="center">3</td>
   <td align="center">Gini</td>
   <td>0.8</td>
   <td>0.67</td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Gini</td>
   <td>0.91</td>
   <td>0.67</td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Gini</td>
   <td>1.0</td>
   <td>0.67</td>
   </tr>
   <tr>
   <td align="center">3</td>
   <td align="center">Entropía</td>
   <td>0.78</td>
   <td>0.64</td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Entropía</td>
   <td>0.89</td>
   <td>0.64</td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Entropía</td>
   <td>1.0</td>
   <td>0.63</td>
   </tr>
   </tbody></table>

conclusiones: (a checkear si estan bien calculadas las métricas, siento que tendrian que dar otra cosa) Podemos observar que un aumento en la altura máxima de los arboles no se condice con una mejora en la performance en la etapa de validacion. Esto se debe a que si los arboles tienen altura infinita van a asegurarse de que podamos clasificar correctamente a todas las instancias de train (por eso accuracy de train aumenta a medida que incrementamos la altura máxima); sin embargo esto nos deja en un claro caso de overfitting, lo cual se evidencia al ver que a pesar de una mejora sustancial en la accuracy de train, la accuracy de validation no solo no mejora sino que empeora. 
A modo de conclusion podemos afirmar que aumentar la altura maxima de los arboles no solo va a resultar mas costoso computacionalmente sino que va a terminar por empeorar nuestro algoritmo.

# 3) Comparación de algoritmos con RandomizedSearchCV

Algoritmos a probar:

* Árboles de decisión (esto ya lo hicimos recien, buscamos mejores arboles con hiperparametros? Si, aca probamos mas hp)
* KNN (k-vecinos más cercanos)
* SVM (Support vector machine)
* LDA (Linear discriminant analysis)
* Naïve Bayes

podemos tomar el test score global como el promedio de cada uno de los folds (esta mal pero no tan mal)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

def fine_tune(X, y, classifier, model_name, grid, cv, scoring, objective, random=True):
    print('\nFine tuning {0}. Objective: {1}'.format(model_name, objective))

    if random:
      random_search = RandomizedSearchCV(estimator=classifier, param_distributions=grid, n_jobs=-1, cv=cv, scoring=scoring, refit=objective)
      random_result = random_search.fit(X, y)
      print("Best score was : %f using %s" % (random_result.best_score_, random_result.best_params_))
      return random_result.best_estimator_
    else:
      # TODO borrar si estan de acuerdo, total dice q no se puede usar
      grid_search = GridSearchCV(estimator=classifier, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit=objective)
      grid_result = grid_search.fit(X, y)
      print("Best score was : %f using %s" % (grid_result.best_score_, grid_result.best_params_))
      return grid_result.best_estimator_

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from scipy.stats import uniform, randint

# Se iniciailizan default porque despues se van variando los hiperparámetros
classifiers_to_test = {
    'Árbol de decisión': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(n_jobs=-1),
    'SVM' : SVC(),
    'LDA' : LDA(),
    'Naïve Bayes': GaussianNB()
}

grids = dict()

grids['Árbol de decisión'] = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 50, 100, 150, 200],
    'max_features': ['sqrt', 'log2', None]
}
grids['KNN'] = {
    'n_neighbors': range(5, 26, 5),
    'weights': ['uniform', 'distance'],
    'leaf_size': randint(20, 50),
    'p': [1, 2]
}
grids['SVM'] = {
    'C': [0.5, 1, 2],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [1, 2, 3]
}
grids['LDA'] = {
    'solver': ['lsqr', 'eigen'],
    'shrinkage': [None, 'auto', 0, 0.25, 0.5, 0.75, 1]
}
grids['Naïve Bayes'] = {
    'var_smoothing': uniform(1e-9, 1e-7)
}

# TODO Ver estos warnings jeje
import warnings
warnings.filterwarnings("ignore")

best_classifier = dict()
for clf_name, clf in classifiers_to_test.items():
    # fine_tune devuelve el estimador con la mejor combinación entre las que se prueban, en el dict guardamos la mejor config para cada algoritmo
    best_classifier[clf_name] = fine_tune(X_dev, y_dev, clf, clf_name, grids[clf_name], balanced_k_fold, scoring, objective='roc_auc')



Fine tuning Árbol de decisión. Objective: roc_auc
Best score was : 0.609554 using {'splitter': 'random', 'max_features': 'log2', 'max_depth': 50, 'criterion': 'gini'}

Fine tuning KNN. Objective: roc_auc
Best score was : 0.686973 using {'leaf_size': 46, 'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}

Fine tuning SVM. Objective: roc_auc
Best score was : 0.708551 using {'kernel': 'rbf', 'degree': 2, 'C': 1}

Fine tuning LDA. Objective: roc_auc
Best score was : 0.743834 using {'solver': 'lsqr', 'shrinkage': 0.5}

Fine tuning Naïve Bayes. Objective: roc_auc
Best score was : 0.674445 using {'var_smoothing': 2.2191999701110953e-08}


In [ ]:
best_classifier

{'Árbol de decisión': DecisionTreeClassifier(max_depth=50, max_features='log2', splitter='random'),
 'KNN': KNeighborsClassifier(leaf_size=46, n_jobs=-1, p=1),
 'SVM': SVC(C=1, degree=2),
 'LDA': LinearDiscriminantAnalysis(shrinkage=0.5, solver='lsqr'),
 'Naïve Bayes': GaussianNB(var_smoothing=2.2191999701110953e-08)}

Construir un modelo RandomForest con 200 árboles. Explorar para qué sirve el hiperparámetro max_features y cómo afecta a la performance del algoritmo mediante una curva de complejidad. Explicar por qué creen que se dieron los resultados obtenidos. Por último, graficar una curva de aprendizaje sobre los parámetros elegidos para determinar si sería útil o no conseguir más datos.

max features = The number of features to consider when looking for the best split // 
n_estimators = cantidad de árboles 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier(randomforest = RandomForestClassifier(n_estimators = 200))

In [ ]:
# quizas sea util al final


import matplotlib.pyplot as plt

def youden(fpr, tpr, thresholds):
    optimal_idx = np.argmax(tpr - fpr)
    return thresholds[optimal_idx]

def validate_classifiers(classifiers, X_train, y_train, X_test, y_test):
    for name, clf in classifiers.items():
        print('Validation for {0}'.format(name))
        pipeline_clf = make_pipeline(preprocessing.StandardScaler(), clf)
        pipeline_clf.fit(X_train, y_train)
        metrics.plot_roc_curve(pipeline_clf, X_test, y_test)
        churn_probas = pipeline_clf.predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds = metrics.roc_curve(y_test, churn_probas)
        youden_score = youden(fpr, tpr, thresholds)
        print('Youden spot ---> {0}'.format(youden_score))
        plt.show()